##**Gerrymandering-Environment**

    INITIAL STATE (provided externally via reset(options=...)):
        - 'district_map'
        - 'social_graph'
        - 'opinions'     

    ACTION:
        - new district assignment for each voter

    OBSERVATION (returned by reset/step):
        {
          'district_map'   : (num_voters,)
          'representatives': (num_districts,)  # voter indices; -1 if empty district
          'social_graph'   : (num_voters, num_voters)  # AUGMENTED: base social + rep->voter edges used for the step
          'opinions'       : (num_voters, 2)
          'opinion_graph'  : (num_voters, num_voters)  # similarity kernel derived from opinion distances
        }

    KEY LOGIC:
      - Representatives: for each district, pick the member that minimizes the sum of L2 distances to members in that district (discrete 1-median).
      - Opinion dynamics: DRF (assimilation/neutral/backfire) with weighted neighbor influence.
      - Reward: reduction in total distance to reference opinion c*

    Notes:
      - Opinion weight = 1
      - Opinion dimension is fixed at 2.
      - We accept any districting action

In [1]:
!pip install torch_geometric

In [2]:
import numpy as np
import torch
from torch_geometric.data import Data
import gymnasium as gym
from gymnasium import spaces
import gerry_environment

#Inch Worm Test

In [3]:
# test data based on the inchworm example in the paper
import itertools

num_voters = 10
num_districts = 1
T = 8
triple_size = 3
opinion_dim = 1

initial_opinions = np.array([[0,0,0,1,2,3,4,5,5,5]], dtype=float).reshape(num_voters,opinion_dim)
traget_opinions = initial_opinions+1
traget_opinions = traget_opinions.flatten()

pos = np.arange(num_voters)[:,None]
pos = np.hstack([pos, np.zeros_like(pos)])

edges = []
edge_index = np.empty((2,0), dtype=int)
edge_attr = np.empty((0,), dtype=float)
# edges = [(i,i+1) for i in range(num_voters-1)]
# edge_index = np.array(edges+[(j,i) for i,j in edges]).T
# edge_attr = np.ones(edge_index.shape[1])
triple_sequence = [
    (0, 3, 6),
    (0, 4, 6),
    (1, 5, 7),
    (2, 4, 8),
    (3, 5, 9),
    (0, 1, 2),
    (3, 4, 5),
    (3, 4, 5),
]

env = gerry_environment.FrankenmanderingEnv(num_voters, num_districts,opinion_dim,8,0, traget_opinions)
obs,_=env.reset(options={"opinions":initial_opinions,"pos":pos,"edge_index":edge_index,"edge_attr":edge_attr})

y_cur = env._assignment.argmax(axis=1).astype(np.int32)

In [4]:
for t in range(T):
    X = env._x.copy()
    best_gain = -np.inf
    best_trip = None
    best_A = None

    # try all possible triples of voters
    for trip in itertools.combinations(range(num_voters), 3):
        # build assignment: one triple, rest singletons
        remaining = [v for v in range(num_voters) if v not in trip]   # keeps deterministic increasing order
        groups = [list(trip)] + [[v] for v in remaining]

        num_groups = len(groups)                    # = 1 + (num_voters - 3)
        A = np.zeros((num_voters, num_groups))      # ensure shape matches actual groups
        for j, group in enumerate(groups):
            A[group, j] = 1.0

        # simulate update
        y = A.argmax(axis=1).astype(np.int64)
        reps = env._elect_representatives_from_labels(y, X)
        edge_index_aug, edge_attr_aug = env._augment_with_reps(env._edge_index, env._edge_attr, reps, y)
        X_new = env._opinion_update(edge_index_aug, edge_attr_aug, X, eta=1.0)
        gain = env._reward(X, X_new)

        # check if this triple is better

        if gain > best_gain:
            best_gain = gain
            best_trip = trip
            best_A = A
    if t==0:
      print('best_gain=',best_gain)
    if best_gain <0:
          print(f"t={t+1}, no improving triple found, stopping.")
          break
    # commit the best action
    obs, reward, terminated, truncated, info = env.step(best_A)
    print(f"t={t+1}, chosen triple={best_trip}, reward={reward}")
    print("opinions:", env._x.flatten())
    if terminated:
        break


best_gain= 2.0
t=1, chosen triple=(0, 3, 6), reward=2.0
opinions: [1. 0. 0. 1. 2. 3. 5. 5. 5. 5.]
t=2, chosen triple=(0, 1, 7), reward=2.0
opinions: [1. 1. 0. 1. 2. 3. 5. 6. 5. 5.]
t=3, chosen triple=(0, 2, 8), reward=2.0
opinions: [1. 1. 1. 1. 2. 3. 5. 6. 6. 5.]
t=4, chosen triple=(3, 4, 9), reward=2.0
opinions: [1. 1. 1. 2. 2. 3. 5. 6. 6. 6.]
t=5, chosen triple=(0, 1, 2), reward=0.0
opinions: [1. 1. 1. 2. 2. 3. 5. 6. 6. 6.]
t=6, chosen triple=(0, 1, 2), reward=0.0
opinions: [1. 1. 1. 2. 2. 3. 5. 6. 6. 6.]
t=7, chosen triple=(0, 1, 2), reward=0.0
opinions: [1. 1. 1. 2. 2. 3. 5. 6. 6. 6.]
t=8, chosen triple=(0, 1, 2), reward=0.0
opinions: [1. 1. 1. 2. 2. 3. 5. 6. 6. 6.]
